In [173]:
#Whole Numbers, 3 Targets
import pandas as pd
import numpy as np
import tensorflow
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
import datetime as dt

In [95]:
walmart = pd.read_csv('walmartdata.csv')
walmart.head()

,Unnamed: 0,ID,Store,Start_of_Week,Week_Date,Fuel_Price,CPI,Unemployment,Type,Size,Week,Year,Temperature_C,Holiday_Name,Weekly_Sales
0,0,0,1,2010-01-30,2010-02-05,2.57,211.0964,8.106,A,151315,5,2010,6.0,No Holiday,1643690.90
1,1,1,1,2010-02-06,2010-02-12,2.55,211.2422,8.106,A,151315,6,2010,4.0,No Holiday,1641957.44
2,2,2,1,2010-02-13,2010-02-19,2.51,211.2891,8.106,A,151315,7,2010,4.0,No Holiday,1611968.17
3,3,3,1,2010-02-20,2010-02-26,2.56,211.3196,8.106,A,151315,8,2010,8.0,No Holiday,1409727.59
4,4,4,1,2010-02-27,2010-03-05,2.62,211.3501,8.106,A,151315,9,2010,8.0,No Holiday,1554806.68


In [96]:
totalave = walmart.groupby(["Store"])["Weekly_Sales"].mean()

In [97]:
for i in range(len(walmart)):
    walmart.loc[i, "Ave_Sales"] = totalave[walmart.loc[i, "Store"]]
walmart

,Unnamed: 0,ID,Store,Start_of_Week,Week_Date,Fuel_Price,CPI,Unemployment,Type,Size,Week,Year,Temperature_C,Holiday_Name,Weekly_Sales,Ave_Sales
0,0,0,1,2010-01-30,2010-02-05,2.57,211.0964,8.106,A,151315,5,2010,6.0,No Holiday,1643690.90,1.555264e+06
1,1,1,1,2010-02-06,2010-02-12,2.55,211.2422,8.106,A,151315,6,2010,4.0,No Holiday,1641957.44,1.555264e+06
2,2,2,1,2010-02-13,2010-02-19,2.51,211.2891,8.106,A,151315,7,2010,4.0,No Holiday,1611968.17,1.555264e+06
3,3,3,1,2010-02-20,2010-02-26,2.56,211.3196,8.106,A,151315,8,2010,8.0,No Holiday,1409727.59,1.555264e+06
4,4,4,1,2010-02-27,2010-03-05,2.62,211.3501,8.106,A,151315,9,2010,8.0,No Holiday,1554806.68,1.555264e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6434,6434,6434,45,2012-09-22,2012-09-28,4.00,192.0136,8.684,B,118221,39,2012,18.0,No Holiday,713173.95,7.859814e+05
6435,6435,6435,45,2012-09-29,2012-10-05,3.98,192.1704,8.667,B,118221,40,2012,18.0,No Holiday,733455.07,7.859814e+05
6436,6436,6436,45,2012-10-06,2012-10-12,4.00,192.3273,8.667,B,118221,41,2012,12.0,No Holiday,734464.36,7.859814e+05
6437,6437,6437,45,2012-10-13,2012-10-19,3.97,192.3309,8.667,B,118221,42,2012,14.0,No Holiday,718125.53,7.859814e+05


In [98]:
walmart = walmart.filter(items=['Fuel_Price', "CPI", "Unemployment", "Temperature_C", "Weekly_Sales","Ave_Sales"])

In [99]:
walmart.loc[walmart['Weekly_Sales'] < (0.85 * walmart["Ave_Sales"]), 'Label'] = 'Below' 
walmart.loc[walmart['Weekly_Sales'] > (1.15 * walmart["Ave_Sales"]), 'Label'] = 'Above' 
walmart.loc[(walmart['Weekly_Sales'] > (0.85 * walmart["Ave_Sales"])) & (walmart['Weekly_Sales'] < (1.15 * walmart["Ave_Sales"])), 'Label'] = 'Average'

In [100]:
walmart = walmart.filter(items=['Fuel_Price', "CPI", "Unemployment", "Temperature_C", "Label"])

In [101]:
walmart["Label"].value_counts()

Average    5601
Above       500
Below       338
Name: Label, dtype: int64

In [102]:
X = walmart.drop("Label", axis=1)
y = walmart["Label"]
print(X.shape, y.shape)

(6439, 4) (6439,)


In [103]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [104]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

# Scale and train x value datasets
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [105]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [13]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=42)
model

LogisticRegression(random_state=42)

In [375]:
model.fit(X_train_scaled, encoded_y_train)

LogisticRegression(random_state=42)

In [376]:
print(f"Training Data Score: {model.score(X_train_scaled, encoded_y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, encoded_y_test)}")

Training Data Score: 0.8709877821495133
Testing Data Score: 0.8664596273291926


In [377]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 50],
              'solver': ['newton-cg', 'liblinear', 'sag', 'saga'],
             'max_iter':[200, 500, 1000]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [378]:
grid.fit(X_train_scaled, encoded_y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] C=1, max_iter=200, solver=newton-cg .............................
[CV] . C=1, max_iter=200, solver=newton-cg, score=0.872, total=   0.1s
[CV] C=1, max_iter=200, solver=newton-cg .............................
[CV] . C=1, max_iter=200, solver=newton-cg, score=0.871, total=   0.0s
[CV] C=1, max_iter=200, solver=newton-cg .............................
[CV] . C=1, max_iter=200, solver=newton-cg, score=0.871, total=   0.0s
[CV] C=1, max_iter=200, solver=newton-cg .............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


[CV] . C=1, max_iter=200, solver=newton-cg, score=0.871, total=   0.1s
[CV] C=1, max_iter=200, solver=newton-cg .............................
[CV] . C=1, max_iter=200, solver=newton-cg, score=0.872, total=   0.0s
[CV] C=1, max_iter=200, solver=liblinear .............................
[CV] . C=1, max_iter=200, solver=liblinear, score=0.872, total=   0.2s
[CV] C=1, max_iter=200, solver=liblinear .............................
[CV] . C=1, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=1, max_iter=200, solver=liblinear .............................
[CV] . C=1, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=1, max_iter=200, solver=liblinear .............................
[CV] . C=1, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=1, max_iter=200, solver=liblinear .............................
[CV] . C=1, max_iter=200, solver=liblinear, score=0.872, total=   0.0s
[CV] C=1, max_iter=200, solver=sag ...................................
[CV] .

[CV] . C=5, max_iter=200, solver=newton-cg, score=0.871, total=   0.1s
[CV] C=5, max_iter=200, solver=newton-cg .............................
[CV] . C=5, max_iter=200, solver=newton-cg, score=0.871, total=   0.1s
[CV] C=5, max_iter=200, solver=newton-cg .............................
[CV] . C=5, max_iter=200, solver=newton-cg, score=0.871, total=   0.0s
[CV] C=5, max_iter=200, solver=newton-cg .............................
[CV] . C=5, max_iter=200, solver=newton-cg, score=0.872, total=   0.0s
[CV] C=5, max_iter=200, solver=liblinear .............................
[CV] . C=5, max_iter=200, solver=liblinear, score=0.872, total=   0.0s
[CV] C=5, max_iter=200, solver=liblinear .............................
[CV] . C=5, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=5, max_iter=200, solver=liblinear .............................
[CV] . C=5, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=5, max_iter=200, solver=liblinear .............................
[CV] .

[CV]  C=10, max_iter=200, solver=newton-cg, score=0.871, total=   0.1s
[CV] C=10, max_iter=200, solver=newton-cg ............................
[CV]  C=10, max_iter=200, solver=newton-cg, score=0.871, total=   0.0s
[CV] C=10, max_iter=200, solver=newton-cg ............................
[CV]  C=10, max_iter=200, solver=newton-cg, score=0.872, total=   0.0s
[CV] C=10, max_iter=200, solver=liblinear ............................
[CV]  C=10, max_iter=200, solver=liblinear, score=0.872, total=   0.0s
[CV] C=10, max_iter=200, solver=liblinear ............................
[CV]  C=10, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=10, max_iter=200, solver=liblinear ............................
[CV]  C=10, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=10, max_iter=200, solver=liblinear ............................
[CV]  C=10, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=10, max_iter=200, solver=liblinear ............................
[CV]  

[CV]  C=50, max_iter=200, solver=newton-cg, score=0.871, total=   0.1s
[CV] C=50, max_iter=200, solver=newton-cg ............................
[CV]  C=50, max_iter=200, solver=newton-cg, score=0.871, total=   0.1s
[CV] C=50, max_iter=200, solver=newton-cg ............................
[CV]  C=50, max_iter=200, solver=newton-cg, score=0.872, total=   0.1s
[CV] C=50, max_iter=200, solver=liblinear ............................
[CV]  C=50, max_iter=200, solver=liblinear, score=0.872, total=   0.0s
[CV] C=50, max_iter=200, solver=liblinear ............................
[CV]  C=50, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=50, max_iter=200, solver=liblinear ............................
[CV]  C=50, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=50, max_iter=200, solver=liblinear ............................
[CV]  C=50, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=50, max_iter=200, solver=liblinear ............................
[CV]  

[Parallel(n_jobs=1)]: Done 240 out of 240 | elapsed:    6.3s finished


GridSearchCV(estimator=LogisticRegression(random_state=42),
             param_grid={'C': [1, 5, 10, 50], 'max_iter': [200, 500, 1000],
                         'solver': ['newton-cg', 'liblinear', 'sag', 'saga']},
             verbose=3)

In [379]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 1, 'max_iter': 200, 'solver': 'newton-cg'}
0.8709878887351291


In [382]:
logreg = LogisticRegression(C=1, max_iter=200, solver= 'newton-cg', random_state=42)
logreg.fit(X_train_scaled, encoded_y_train)
print(f"Training Data Score: {logreg.score(X_train_scaled, encoded_y_train)}")
print(f"Testing Data Score: {logreg.score(X_test_scaled, encoded_y_test)}")

Training Data Score: 0.8709877821495133
Testing Data Score: 0.8664596273291926


In [32]:
target = walmart["Label"]
target_names = ["Above", "Average", "Below"]

In [31]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000, random_state=42)
rf = rf.fit(X_train_scaled, encoded_y_train)
rf.score(X_test_scaled, encoded_y_test)

0.8813664596273292

In [33]:
feature_names = walmart.columns
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.3311428636926653, 'CPI'),
 (0.2694866326503375, 'Fuel_Price'),
 (0.20966494957819504, 'Temperature_C'),
 (0.18970555407880219, 'Unemployment')]

In [385]:
param_grid = {'criterion': ['gini', 'entropy'],
              'splitter': ['best', 'random'],
             'class_weight': ['None', 'balanced']}
grid = GridSearchCV(rf, param_grid, verbose=3)

In [386]:
grid.fit(X_train_scaled, encoded_y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=random ..............
[CV]  class_weight=None, criterion=gini, splitter=random, score=nan, total=   0.0s
[CV] class_weight=None, cr

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/tree/_classes.py", line 890, in fit
    super().fit(
  File "/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/tree/_classes.py", line 199, in fit
    expanded_class_weight = compute_sample_weight(
  File "/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py", line 73, in inner_f
    return f(**kwargs)
  File "/Users/ronal

[CV]  class_weight=balanced, criterion=entropy, splitter=best, score=0.851, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.829, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.827, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.836, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.839, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.829, total=   0.0s


[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.2s finished


GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'class_weight': ['None', 'balanced'],
                         'criterion': ['gini', 'entropy'],
                         'splitter': ['best', 'random']},
             verbose=3)

In [387]:
print(grid.best_params_)
print(grid.best_score_)

{'class_weight': 'balanced', 'criterion': 'entropy', 'splitter': 'best'}
0.8482092706422512


In [106]:
ranfor = RandomForestClassifier(n_estimators=500, random_state=42)
ranfor = ranfor.fit(X_train_scaled, encoded_y_train)
ranfor.score(X_test_scaled, encoded_y_test)

0.8826086956521739

In [107]:
clf = tree.DecisionTreeClassifier(random_state=42)
clf = clf.fit(X_train_scaled, encoded_y_train)
clf.score(X_test_scaled, encoded_y_test)

0.8540372670807453

In [36]:
feature_names = walmart.columns
sorted(zip(clf.feature_importances_, feature_names), reverse=True)

[(0.36486851439318435, 'CPI'),
 (0.27004417922651425, 'Fuel_Price'),
 (0.182869784976228, 'Unemployment'),
 (0.1822175214040735, 'Temperature_C')]

In [391]:
param_grid = {'criterion': ['gini', 'entropy'],
              'splitter': ['best', 'random'],
             'class_weight': ['None', 'balanced']}
grid = GridSearchCV(clf, param_grid, verbose=3)

In [392]:
grid.fit(X_train_scaled, encoded_y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=random ..............
[CV]  class_weight=None, criterion=gini, splitter=random, score=nan, total=   0.0s
[CV] class_weight=None, cr

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/tree/_classes.py", line 890, in fit
    super().fit(
  File "/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/tree/_classes.py", line 199, in fit
    expanded_class_weight = compute_sample_weight(
  File "/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py", line 73, in inner_f
    return f(**kwargs)
  File "/Users/ronal

[CV]  class_weight=balanced, criterion=entropy, splitter=best, score=0.851, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.829, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.827, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.836, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.839, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.829, total=   0.0s


[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.2s finished


GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'class_weight': ['None', 'balanced'],
                         'criterion': ['gini', 'entropy'],
                         'splitter': ['best', 'random']},
             verbose=3)

In [393]:
print(grid.best_params_)
print(grid.best_score_)

{'class_weight': 'balanced', 'criterion': 'entropy', 'splitter': 'best'}
0.8482092706422512


In [108]:
dectre = tree.DecisionTreeClassifier(random_state=42)
dectre = dectre.fit(X_train_scaled, encoded_y_train)
dectre.score(X_test_scaled, encoded_y_test)

0.8540372670807453

In [109]:
dectre_predictions = dectre.predict(X_test_scaled)

In [110]:
ranfor_predictions = ranfor.predict(X_test_scaled)

In [111]:
print(classification_report(encoded_y_test, ranfor_predictions,
                            target_names= target_names))

              precision    recall  f1-score   support

       Above       0.71      0.45      0.55       134
     Average       0.91      0.96      0.94      1395
       Below       0.38      0.25      0.30        81

    accuracy                           0.88      1610
   macro avg       0.67      0.55      0.59      1610
weighted avg       0.87      0.88      0.87      1610



In [112]:
print(classification_report(encoded_y_test, dectre_predictions,
                            target_names= target_names))

              precision    recall  f1-score   support

       Above       0.54      0.57      0.56       134
     Average       0.92      0.91      0.92      1395
       Below       0.29      0.30      0.29        81

    accuracy                           0.85      1610
   macro avg       0.58      0.59      0.59      1610
weighted avg       0.86      0.85      0.86      1610



In [202]:
predix = pd.read_csv('prophet_predictions.csv')
predix = predix.drop(['Unnamed: 0'], axis=1)
predix.head()

,Date,Store,Fuel_Price,CPI,Unemployment,Temperature_C,Weekly_Sales
0,2012-10-21,1,3.487440,223.461145,6.627970,17.659895,1.529256e+06
1,2012-10-28,1,3.472333,223.443172,6.634452,16.448992,1.587566e+06
2,2012-11-04,1,3.456623,223.409742,6.623966,15.104578,1.678286e+06
3,2012-11-11,1,3.441573,223.372609,6.592424,13.620766,1.742059e+06
4,2012-11-18,1,3.423539,223.358492,6.556789,12.028795,1.765627e+06


In [203]:
predix = predix.filter(items=['Date', 'Store', 'Fuel_Price', "CPI", "Unemployment", "Temperature_C"])

In [204]:
predix

,Date,Store,Fuel_Price,CPI,Unemployment,Temperature_C
0,2012-10-21,1,3.487440,223.461145,6.627970,17.659895
1,2012-10-28,1,3.472333,223.443172,6.634452,16.448992
2,2012-11-04,1,3.456623,223.409742,6.623966,15.104578
3,2012-11-11,1,3.441573,223.372609,6.592424,13.620766
4,2012-11-18,1,3.423539,223.358492,6.556789,12.028795
...,...,...,...,...,...,...
2470,2013-10-06,45,4.393188,195.136141,8.878137,16.570370
2471,2013-10-13,45,4.400800,195.177228,8.872678,15.371153
2472,2013-10-20,45,4.394871,195.161653,8.877326,13.409695
2473,2013-10-27,45,4.380892,195.115184,8.887845,11.328064


In [205]:
predix
X = predix.drop("Date", axis=1)
X = X.drop("Store", axis=1)
X

,Fuel_Price,CPI,Unemployment,Temperature_C
0,3.487440,223.461145,6.627970,17.659895
1,3.472333,223.443172,6.634452,16.448992
2,3.456623,223.409742,6.623966,15.104578
3,3.441573,223.372609,6.592424,13.620766
4,3.423539,223.358492,6.556789,12.028795
...,...,...,...,...
2470,4.393188,195.136141,8.878137,16.570370
2471,4.400800,195.177228,8.872678,15.371153
2472,4.394871,195.161653,8.877326,13.409695
2473,4.380892,195.115184,8.887845,11.328064


In [206]:
predix_scaled = X_scaler.transform(X)

In [207]:
ranfor_labels = ranfor.predict(predix_scaled)

In [208]:
ranfor_labels = label_encoder.inverse_transform(ranfor_labels)
ranfor_labels

array(['Average', 'Average', 'Average', ..., 'Average', 'Average',
       'Average'], dtype=object)

In [120]:
ranfor_predix = predix
ranfor_predix["Label"] = ranfor_labels
ranfor_predix

,Date,Store,Fuel_Price,CPI,Unemployment,Temperature_C,Label
0,2012-10-21,1,3.487440,223.461145,6.627970,17.659895,Average
1,2012-10-28,1,3.472333,223.443172,6.634452,16.448992,Average
2,2012-11-04,1,3.456623,223.409742,6.623966,15.104578,Average
3,2012-11-11,1,3.441573,223.372609,6.592424,13.620766,Average
4,2012-11-18,1,3.423539,223.358492,6.556789,12.028795,Average
...,...,...,...,...,...,...,...
2470,2013-10-06,45,4.393188,195.136141,8.878137,16.570370,Average
2471,2013-10-13,45,4.400800,195.177228,8.872678,15.371153,Average
2472,2013-10-20,45,4.394871,195.161653,8.877326,13.409695,Average
2473,2013-10-27,45,4.380892,195.115184,8.887845,11.328064,Average


In [121]:
ranfor_predix["Label"].value_counts()

Average    2435
Below        22
Above        18
Name: Label, dtype: int64

In [209]:
dectre_labels = dectre.predict(predix_scaled)

In [210]:
dectre_labels = label_encoder.inverse_transform(dectre_labels)
dectre_labels

array(['Average', 'Average', 'Average', ..., 'Average', 'Average',
       'Average'], dtype=object)

In [211]:
dectre_predix = predix
dectre_predix["Label"] = dectre_labels
dectre_predix

,Date,Store,Fuel_Price,CPI,Unemployment,Temperature_C,Label
0,2012-10-21,1,3.487440,223.461145,6.627970,17.659895,Average
1,2012-10-28,1,3.472333,223.443172,6.634452,16.448992,Average
2,2012-11-04,1,3.456623,223.409742,6.623966,15.104578,Average
3,2012-11-11,1,3.441573,223.372609,6.592424,13.620766,Average
4,2012-11-18,1,3.423539,223.358492,6.556789,12.028795,Average
...,...,...,...,...,...,...,...
2470,2013-10-06,45,4.393188,195.136141,8.878137,16.570370,Average
2471,2013-10-13,45,4.400800,195.177228,8.872678,15.371153,Average
2472,2013-10-20,45,4.394871,195.161653,8.877326,13.409695,Average
2473,2013-10-27,45,4.380892,195.115184,8.887845,11.328064,Average


In [127]:
dectre_predix["Label"].value_counts()

Average    2095
Above       242
Below       138
Name: Label, dtype: int64

In [457]:
predix

,Date,Store,Fuel_Diff,CPI_Diff,Unem_Diff,Temp_Diff,Label
0,2012-10-21,1,0.267649,7.464254,-0.982450,-2.507938,Average
1,2012-10-28,1,0.252542,7.446281,-0.975968,-3.718840,Average
2,2012-11-04,1,0.236833,7.412851,-0.986453,-5.063254,Average
3,2012-11-11,1,0.221783,7.375718,-1.017996,-6.547066,Average
4,2012-11-18,1,0.203749,7.361601,-1.053631,-8.139037,Average
...,...,...,...,...,...,...,...
2470,2013-10-06,45,0.975846,8.850462,0.229388,2.227712,Above
2471,2013-10-13,45,0.983458,8.891549,0.223930,1.028496,Above
2472,2013-10-20,45,0.977529,8.875975,0.228578,-0.932962,Average
2473,2013-10-27,45,0.963550,8.829505,0.239096,-3.014594,Average


In [212]:
predix = pd.read_csv('prophet_predictions.csv')

In [213]:
for i in range(len(predix)):
    predix.loc[i, "Ave_Sales"] = totalave[predix.loc[i, "Store"]]
predix

,Unnamed: 0,Date,Store,Fuel_Price,CPI,Unemployment,Temperature_C,Weekly_Sales,Ave_Sales
0,0,2012-10-21,1,3.487440,223.461145,6.627970,17.659895,1.529256e+06,1.555264e+06
1,1,2012-10-28,1,3.472333,223.443172,6.634452,16.448992,1.587566e+06,1.555264e+06
2,2,2012-11-04,1,3.456623,223.409742,6.623966,15.104578,1.678286e+06,1.555264e+06
3,3,2012-11-11,1,3.441573,223.372609,6.592424,13.620766,1.742059e+06,1.555264e+06
4,4,2012-11-18,1,3.423539,223.358492,6.556789,12.028795,1.765627e+06,1.555264e+06
...,...,...,...,...,...,...,...,...,...
2470,2470,2013-10-06,45,4.393188,195.136141,8.878137,16.570370,7.524903e+05,7.859814e+05
2471,2471,2013-10-13,45,4.400800,195.177228,8.872678,15.371153,7.420480e+05,7.859814e+05
2472,2472,2013-10-20,45,4.394871,195.161653,8.877326,13.409695,7.451909e+05,7.859814e+05
2473,2473,2013-10-27,45,4.380892,195.115184,8.887845,11.328064,7.835365e+05,7.859814e+05


In [214]:
predix.loc[predix['Weekly_Sales'] < (0.85 * predix["Ave_Sales"]), 'Label'] = 'Below' 
predix.loc[predix['Weekly_Sales'] > (1.15 * predix["Ave_Sales"]), 'Label'] = 'Above' 
predix.loc[(predix['Weekly_Sales'] > (0.85 * predix["Ave_Sales"])) & (predix['Weekly_Sales'] < (1.15 * predix["Ave_Sales"])), 'Label'] = 'Average'
proph_predix = predix

In [215]:
proph_predix["Label"].value_counts()

Average    1888
Above       404
Below       183
Name: Label, dtype: int64

In [216]:
proph_predix = proph_predix.rename(columns={"Label": "Proph_Label"})

In [217]:
dectre_predix["Label"].value_counts()

Average    2095
Above       242
Below       138
Name: Label, dtype: int64

In [218]:
dectre_predix

,Date,Store,Fuel_Price,CPI,Unemployment,Temperature_C,Label
0,2012-10-21,1,3.487440,223.461145,6.627970,17.659895,Average
1,2012-10-28,1,3.472333,223.443172,6.634452,16.448992,Average
2,2012-11-04,1,3.456623,223.409742,6.623966,15.104578,Average
3,2012-11-11,1,3.441573,223.372609,6.592424,13.620766,Average
4,2012-11-18,1,3.423539,223.358492,6.556789,12.028795,Average
...,...,...,...,...,...,...,...
2470,2013-10-06,45,4.393188,195.136141,8.878137,16.570370,Average
2471,2013-10-13,45,4.400800,195.177228,8.872678,15.371153,Average
2472,2013-10-20,45,4.394871,195.161653,8.877326,13.409695,Average
2473,2013-10-27,45,4.380892,195.115184,8.887845,11.328064,Average


In [219]:
final = pd.merge(proph_predix, dectre_predix, how="outer", on=["Date", "Store", "Fuel_Price", "CPI", "Unemployment", "Temperature_C"])


In [220]:
final = final.drop(['Unnamed: 0'], axis=1)
final

,Date,Store,Fuel_Price,CPI,Unemployment,Temperature_C,Weekly_Sales,Ave_Sales,Proph_Label,Label
0,2012-10-21,1,3.487440,223.461145,6.627970,17.659895,1.529256e+06,1.555264e+06,Average,Average
1,2012-10-28,1,3.472333,223.443172,6.634452,16.448992,1.587566e+06,1.555264e+06,Average,Average
2,2012-11-04,1,3.456623,223.409742,6.623966,15.104578,1.678286e+06,1.555264e+06,Average,Average
3,2012-11-11,1,3.441573,223.372609,6.592424,13.620766,1.742059e+06,1.555264e+06,Average,Average
4,2012-11-18,1,3.423539,223.358492,6.556789,12.028795,1.765627e+06,1.555264e+06,Average,Average
...,...,...,...,...,...,...,...,...,...,...
2470,2013-10-06,45,4.393188,195.136141,8.878137,16.570370,7.524903e+05,7.859814e+05,Average,Average
2471,2013-10-13,45,4.400800,195.177228,8.872678,15.371153,7.420480e+05,7.859814e+05,Average,Average
2472,2013-10-20,45,4.394871,195.161653,8.877326,13.409695,7.451909e+05,7.859814e+05,Average,Average
2473,2013-10-27,45,4.380892,195.115184,8.887845,11.328064,7.835365e+05,7.859814e+05,Average,Average


In [221]:
final.to_csv("final2013predix.csv", index = False, header=True)

In [154]:
pre2012 = pd.read_csv('2012_predictions.csv')

In [155]:
pre2012 = pre2012.drop(['Unnamed: 0'], axis=1)
pre2012.head()

,Date,Fuel_Price,Store,CPI,Unemployment,Temperature_C,Weekly_Sales
0,2012-01-01,3.130357,1,219.768684,7.768308,21.613198,1.604783e+06
1,2012-01-08,3.110403,1,219.979377,7.758391,21.654107,1.606048e+06
2,2012-01-15,3.090448,1,220.190070,7.748474,21.695015,1.607312e+06
3,2012-01-22,3.070493,1,220.400763,7.738557,21.735924,1.608577e+06
4,2012-01-29,3.050539,1,220.611456,7.728640,21.776833,1.609841e+06


In [156]:
pre2012 = pre2012.filter(items=["Date", "Store",'Fuel_Price', "CPI", "Unemployment", "Temperature_C"])

In [157]:
X = pre2012.drop("Date", axis=1)
X = X.drop("Store", axis=1)
X

,Fuel_Price,CPI,Unemployment,Temperature_C
0,3.130357,219.768684,7.768308,21.613198
1,3.110403,219.979377,7.758391,21.654107
2,3.090448,220.190070,7.748474,21.695015
3,3.070493,220.400763,7.738557,21.735924
4,3.050539,220.611456,7.728640,21.776833
...,...,...,...,...
2380,2.543061,195.826121,8.124308,19.193262
2381,2.525548,195.963795,8.116957,19.251467
2382,2.508035,196.101468,8.109606,19.309672
2383,2.490522,196.239142,8.102255,19.367877


In [158]:
predix_scaled = X_scaler.transform(X)

In [159]:
dectre_labels = dectre.predict(predix_scaled)

In [160]:
dectre_labels = label_encoder.inverse_transform(dectre_labels)
dectre_labels

array(['Average', 'Above', 'Above', ..., 'Above', 'Above', 'Above'],
      dtype=object)

In [161]:
dectre_predix = pre2012
dectre_predix["Label"] = dectre_labels
dectre_predix

,Date,Store,Fuel_Price,CPI,Unemployment,Temperature_C,Label
0,2012-01-01,1,3.130357,219.768684,7.768308,21.613198,Average
1,2012-01-08,1,3.110403,219.979377,7.758391,21.654107,Above
2,2012-01-15,1,3.090448,220.190070,7.748474,21.695015,Above
3,2012-01-22,1,3.070493,220.400763,7.738557,21.735924,Above
4,2012-01-29,1,3.050539,220.611456,7.728640,21.776833,Above
...,...,...,...,...,...,...,...
2380,2012-12-02,45,2.543061,195.826121,8.124308,19.193262,Above
2381,2012-12-09,45,2.525548,195.963795,8.116957,19.251467,Above
2382,2012-12-16,45,2.508035,196.101468,8.109606,19.309672,Above
2383,2012-12-23,45,2.490522,196.239142,8.102255,19.367877,Above


In [162]:
dectre_predix["Label"].value_counts()

Average    1317
Above       795
Below       273
Name: Label, dtype: int64

In [163]:
pre2012 = pd.read_csv('2012_predictions.csv')
for i in range(len(pre2012)):
    pre2012.loc[i, "Ave_Sales"] = totalave[pre2012.loc[i, "Store"]]
pre2012

,Unnamed: 0,Date,Fuel_Price,Store,CPI,Unemployment,Temperature_C,Weekly_Sales,Ave_Sales
0,0,2012-01-01,3.130357,1,219.768684,7.768308,21.613198,1.604783e+06,1.555264e+06
1,1,2012-01-08,3.110403,1,219.979377,7.758391,21.654107,1.606048e+06,1.555264e+06
2,2,2012-01-15,3.090448,1,220.190070,7.748474,21.695015,1.607312e+06,1.555264e+06
3,3,2012-01-22,3.070493,1,220.400763,7.738557,21.735924,1.608577e+06,1.555264e+06
4,4,2012-01-29,3.050539,1,220.611456,7.728640,21.776833,1.609841e+06,1.555264e+06
...,...,...,...,...,...,...,...,...,...
2380,2468,2012-12-02,2.543061,45,195.826121,8.124308,19.193262,8.700074e+05,7.859814e+05
2381,2469,2012-12-09,2.525548,45,195.963795,8.116957,19.251467,8.708043e+05,7.859814e+05
2382,2470,2012-12-16,2.508035,45,196.101468,8.109606,19.309672,8.716012e+05,7.859814e+05
2383,2471,2012-12-23,2.490522,45,196.239142,8.102255,19.367877,8.723981e+05,7.859814e+05


In [164]:
pre2012.loc[pre2012['Weekly_Sales'] < (0.85 * pre2012["Ave_Sales"]), 'Label'] = 'Below' 
pre2012.loc[pre2012['Weekly_Sales'] > (1.15 * pre2012["Ave_Sales"]), 'Label'] = 'Above' 
pre2012.loc[(pre2012['Weekly_Sales'] > (0.85 * pre2012["Ave_Sales"])) & (pre2012['Weekly_Sales'] < (1.15 * pre2012["Ave_Sales"])), 'Label'] = 'Average'
proph_predix = pre2012
proph_predix["Label"].value_counts()

Average    2083
Above       211
Below        91
Name: Label, dtype: int64

In [165]:
dectre_predix["Label"].value_counts()

Average    1317
Above       795
Below       273
Name: Label, dtype: int64

In [166]:
proph_predix = proph_predix.rename(columns={"Label": "Proph_Label"})

In [152]:
dectre_predix

,Fuel_Price,CPI,Unemployment,Temperature_C,Label
0,3.130357,219.768684,7.768308,21.613198,Average
1,3.110403,219.979377,7.758391,21.654107,Above
2,3.090448,220.190070,7.748474,21.695015,Above
3,3.070493,220.400763,7.738557,21.735924,Above
4,3.050539,220.611456,7.728640,21.776833,Above
...,...,...,...,...,...
2380,2.543061,195.826121,8.124308,19.193262,Above
2381,2.525548,195.963795,8.116957,19.251467,Above
2382,2.508035,196.101468,8.109606,19.309672,Above
2383,2.490522,196.239142,8.102255,19.367877,Above


In [153]:
proph_predix

,Unnamed: 0,Date,Fuel_Price,Store,CPI,Unemployment,Temperature_C,Weekly_Sales,Ave_Sales,Proph_Label
0,0,2012-01-01,3.130357,1,219.768684,7.768308,21.613198,1.604783e+06,1.555264e+06,Average
1,1,2012-01-08,3.110403,1,219.979377,7.758391,21.654107,1.606048e+06,1.555264e+06,Average
2,2,2012-01-15,3.090448,1,220.190070,7.748474,21.695015,1.607312e+06,1.555264e+06,Average
3,3,2012-01-22,3.070493,1,220.400763,7.738557,21.735924,1.608577e+06,1.555264e+06,Average
4,4,2012-01-29,3.050539,1,220.611456,7.728640,21.776833,1.609841e+06,1.555264e+06,Average
...,...,...,...,...,...,...,...,...,...,...
2380,2468,2012-12-02,2.543061,45,195.826121,8.124308,19.193262,8.700074e+05,7.859814e+05,Average
2381,2469,2012-12-09,2.525548,45,195.963795,8.116957,19.251467,8.708043e+05,7.859814e+05,Average
2382,2470,2012-12-16,2.508035,45,196.101468,8.109606,19.309672,8.716012e+05,7.859814e+05,Average
2383,2471,2012-12-23,2.490522,45,196.239142,8.102255,19.367877,8.723981e+05,7.859814e+05,Average


In [167]:
final = pd.merge(proph_predix, dectre_predix, how="outer", on=["Date", "Store", "Fuel_Price", "CPI", "Unemployment", "Temperature_C"])

In [168]:
final = final.drop(['Unnamed: 0'], axis=1)

In [169]:
final = final.rename(columns={"Fuel_Price": "Pred_Fuel_Pric", "CPI": "Pred_CPI", "Unemployment": "Pred_Unem", "Temperature_C": "Pred_Temp", 'Weekly_Sales': 'Pred_Week_Sales'})

In [170]:
walmart = pd.read_csv('walmartdata.csv')

In [171]:
walmart['Start_of_Week'] = pd.to_datetime(walmart['Start_of_Week'])
walmart = walmart.loc[ walmart['Start_of_Week'].dt.year == 2012 ]
walmart = walmart.reset_index()
walmart = walmart.drop(['index'], axis=1)
walmart

,Unnamed: 0,ID,Store,Start_of_Week,Week_Date,Fuel_Price,CPI,Unemployment,Type,Size,Week,Year,Temperature_C,Holiday_Name,Weekly_Sales
0,101,101,1,2012-01-07,2012-01-13,3.26,219.8925,7.348,A,151315,2,2012,9.0,No Holiday,1459601.17
1,102,102,1,2012-01-14,2012-01-20,3.27,219.9857,7.348,A,151315,3,2012,12.0,No Holiday,1394393.84
2,103,103,1,2012-01-21,2012-01-27,3.29,220.0789,7.348,A,151315,4,2012,12.0,No Holiday,1319325.59
3,104,104,1,2012-01-28,2012-02-03,3.36,220.1720,7.348,A,151315,5,2012,14.0,No Holiday,1636339.65
4,105,105,1,2012-02-04,2012-02-10,3.41,220.2652,7.348,A,151315,6,2012,9.0,No Holiday,1802477.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,6434,6434,45,2012-09-22,2012-09-28,4.00,192.0136,8.684,B,118221,39,2012,18.0,No Holiday,713173.95
1886,6435,6435,45,2012-09-29,2012-10-05,3.98,192.1704,8.667,B,118221,40,2012,18.0,No Holiday,733455.07
1887,6436,6436,45,2012-10-06,2012-10-12,4.00,192.3273,8.667,B,118221,41,2012,12.0,No Holiday,734464.36
1888,6437,6437,45,2012-10-13,2012-10-19,3.97,192.3309,8.667,B,118221,42,2012,14.0,No Holiday,718125.53


In [174]:
for i in range(len(walmart)):
    walmart.loc[i, "Start_of_Week"] += dt.timedelta(days=1)
walmart

,Unnamed: 0,ID,Store,Start_of_Week,Week_Date,Fuel_Price,CPI,Unemployment,Type,Size,Week,Year,Temperature_C,Holiday_Name,Weekly_Sales
0,101,101,1,2012-01-08,2012-01-13,3.26,219.8925,7.348,A,151315,2,2012,9.0,No Holiday,1459601.17
1,102,102,1,2012-01-15,2012-01-20,3.27,219.9857,7.348,A,151315,3,2012,12.0,No Holiday,1394393.84
2,103,103,1,2012-01-22,2012-01-27,3.29,220.0789,7.348,A,151315,4,2012,12.0,No Holiday,1319325.59
3,104,104,1,2012-01-29,2012-02-03,3.36,220.1720,7.348,A,151315,5,2012,14.0,No Holiday,1636339.65
4,105,105,1,2012-02-05,2012-02-10,3.41,220.2652,7.348,A,151315,6,2012,9.0,No Holiday,1802477.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,6434,6434,45,2012-09-23,2012-09-28,4.00,192.0136,8.684,B,118221,39,2012,18.0,No Holiday,713173.95
1886,6435,6435,45,2012-09-30,2012-10-05,3.98,192.1704,8.667,B,118221,40,2012,18.0,No Holiday,733455.07
1887,6436,6436,45,2012-10-07,2012-10-12,4.00,192.3273,8.667,B,118221,41,2012,12.0,No Holiday,734464.36
1888,6437,6437,45,2012-10-14,2012-10-19,3.97,192.3309,8.667,B,118221,42,2012,14.0,No Holiday,718125.53


In [175]:
final['Date'] = pd.to_datetime(final['Date'])
end_date = walmart['Start_of_Week'].iloc[[-1]].item()
before_end_date = final["Date"] <= end_date
final = final.loc[before_end_date]

final

,Date,Pred_Fuel_Pric,Store,Pred_CPI,Pred_Unem,Pred_Temp,Pred_Week_Sales,Ave_Sales,Proph_Label,Label
0,2012-01-01,3.130357,1,219.768684,7.768308,21.613198,1.604783e+06,1.555264e+06,Average,Average
1,2012-01-08,3.110403,1,219.979377,7.758391,21.654107,1.606048e+06,1.555264e+06,Average,Above
2,2012-01-15,3.090448,1,220.190070,7.748474,21.695015,1.607312e+06,1.555264e+06,Average,Above
3,2012-01-22,3.070493,1,220.400763,7.738557,21.735924,1.608577e+06,1.555264e+06,Average,Above
4,2012-01-29,3.050539,1,220.611456,7.728640,21.776833,1.609841e+06,1.555264e+06,Average,Above
...,...,...,...,...,...,...,...,...,...,...
2370,2012-09-23,2.718193,45,194.449387,8.197820,18.611209,8.620383e+05,7.859814e+05,Average,Average
2371,2012-09-30,2.700680,45,194.587061,8.190469,18.669415,8.628352e+05,7.859814e+05,Average,Average
2372,2012-10-07,2.683167,45,194.724734,8.183118,18.727620,8.636321e+05,7.859814e+05,Average,Average
2373,2012-10-14,2.665653,45,194.862407,8.175767,18.785825,8.644290e+05,7.859814e+05,Average,Average


In [200]:
final.to_csv("final2013.csv", index = False, header=True)

In [176]:
walmart = walmart.rename(columns={"Start_of_Week": "Date"})

In [177]:
comparison = pd.merge(final, walmart, how="outer", on=["Date", "Store"])
comparison = comparison.iloc[1:]
comparison

,Date,Pred_Fuel_Pric,Store,Pred_CPI,Pred_Unem,Pred_Temp,Pred_Week_Sales,Ave_Sales,Proph_Label,Label,...,Fuel_Price,CPI,Unemployment,Type,Size,Week,Year,Temperature_C,Holiday_Name,Weekly_Sales
1,2012-01-08,3.110403,1,219.979377,7.758391,21.654107,1.606048e+06,1.555264e+06,Average,Above,...,3.26,219.8925,7.348,A,151315.0,2.0,2012.0,9.0,No Holiday,1459601.17
2,2012-01-15,3.090448,1,220.190070,7.748474,21.695015,1.607312e+06,1.555264e+06,Average,Above,...,3.27,219.9857,7.348,A,151315.0,3.0,2012.0,12.0,No Holiday,1394393.84
3,2012-01-22,3.070493,1,220.400763,7.738557,21.735924,1.608577e+06,1.555264e+06,Average,Above,...,3.29,220.0789,7.348,A,151315.0,4.0,2012.0,12.0,No Holiday,1319325.59
4,2012-01-29,3.050539,1,220.611456,7.728640,21.776833,1.609841e+06,1.555264e+06,Average,Above,...,3.36,220.1720,7.348,A,151315.0,5.0,2012.0,14.0,No Holiday,1636339.65
5,2012-02-05,3.030584,1,220.822149,7.718724,21.817742,1.611106e+06,1.555264e+06,Average,Above,...,3.41,220.2652,7.348,A,151315.0,6.0,2012.0,9.0,No Holiday,1802477.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1930,2012-09-23,2.718193,45,194.449387,8.197820,18.611209,8.620383e+05,7.859814e+05,Average,Average,...,4.00,192.0136,8.684,B,118221.0,39.0,2012.0,18.0,No Holiday,713173.95
1931,2012-09-30,2.700680,45,194.587061,8.190469,18.669415,8.628352e+05,7.859814e+05,Average,Average,...,3.98,192.1704,8.667,B,118221.0,40.0,2012.0,18.0,No Holiday,733455.07
1932,2012-10-07,2.683167,45,194.724734,8.183118,18.727620,8.636321e+05,7.859814e+05,Average,Average,...,4.00,192.3273,8.667,B,118221.0,41.0,2012.0,12.0,No Holiday,734464.36
1933,2012-10-14,2.665653,45,194.862407,8.175767,18.785825,8.644290e+05,7.859814e+05,Average,Average,...,3.97,192.3309,8.667,B,118221.0,42.0,2012.0,14.0,No Holiday,718125.53


In [178]:
comparison = comparison[["Date", "Store", "Fuel_Price", 'Pred_Fuel_Pric', 'CPI', 'Pred_CPI', 'Temperature_C', 'Pred_Temp', 'Unemployment', 'Pred_Unem', 'Weekly_Sales', "Pred_Week_Sales", 'Ave_Sales','Proph_Label', 'Label']]
comparison

,Date,Store,Fuel_Price,Pred_Fuel_Pric,CPI,Pred_CPI,Temperature_C,Pred_Temp,Unemployment,Pred_Unem,Weekly_Sales,Pred_Week_Sales,Ave_Sales,Proph_Label,Label
1,2012-01-08,1,3.26,3.110403,219.8925,219.979377,9.0,21.654107,7.348,7.758391,1459601.17,1.606048e+06,1.555264e+06,Average,Above
2,2012-01-15,1,3.27,3.090448,219.9857,220.190070,12.0,21.695015,7.348,7.748474,1394393.84,1.607312e+06,1.555264e+06,Average,Above
3,2012-01-22,1,3.29,3.070493,220.0789,220.400763,12.0,21.735924,7.348,7.738557,1319325.59,1.608577e+06,1.555264e+06,Average,Above
4,2012-01-29,1,3.36,3.050539,220.1720,220.611456,14.0,21.776833,7.348,7.728640,1636339.65,1.609841e+06,1.555264e+06,Average,Above
5,2012-02-05,1,3.41,3.030584,220.2652,220.822149,9.0,21.817742,7.348,7.718724,1802477.43,1.611106e+06,1.555264e+06,Average,Above
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1930,2012-09-23,45,4.00,2.718193,192.0136,194.449387,18.0,18.611209,8.684,8.197820,713173.95,8.620383e+05,7.859814e+05,Average,Average
1931,2012-09-30,45,3.98,2.700680,192.1704,194.587061,18.0,18.669415,8.667,8.190469,733455.07,8.628352e+05,7.859814e+05,Average,Average
1932,2012-10-07,45,4.00,2.683167,192.3273,194.724734,12.0,18.727620,8.667,8.183118,734464.36,8.636321e+05,7.859814e+05,Average,Average
1933,2012-10-14,45,3.97,2.665653,192.3309,194.862407,14.0,18.785825,8.667,8.175767,718125.53,8.644290e+05,7.859814e+05,Average,Average


In [179]:
comparison = comparison.rename(columns={"Label": "Model_Label"})

In [180]:
comparison.loc[comparison['Weekly_Sales'] < (0.85 * comparison["Ave_Sales"]), 'Real_Label'] = 'Below' 
comparison.loc[comparison['Weekly_Sales'] > (1.15 * comparison["Ave_Sales"]), 'Real_Label'] = 'Above' 
comparison.loc[(comparison['Weekly_Sales'] > (0.85 * comparison["Ave_Sales"])) & (comparison['Weekly_Sales'] < (1.15 * comparison["Ave_Sales"])), 'Real_Label'] = 'Average'

In [183]:
comparison = comparison.rename(columns={"Model_Label": "Model_Label_Pred"})

In [182]:
comparison["Model_Label"].value_counts()

Average    1134
Above       602
Below       198
Name: Model_Label, dtype: int64

In [184]:
walmart = pd.read_csv('walmartdata.csv')
walmart = walmart.drop(['Unnamed: 0'], axis=1)
walmart.head()

,ID,Store,Start_of_Week,Week_Date,Fuel_Price,CPI,Unemployment,Type,Size,Week,Year,Temperature_C,Holiday_Name,Weekly_Sales
0,0,1,2010-01-30,2010-02-05,2.57,211.0964,8.106,A,151315,5,2010,6.0,No Holiday,1643690.90
1,1,1,2010-02-06,2010-02-12,2.55,211.2422,8.106,A,151315,6,2010,4.0,No Holiday,1641957.44
2,2,1,2010-02-13,2010-02-19,2.51,211.2891,8.106,A,151315,7,2010,4.0,No Holiday,1611968.17
3,3,1,2010-02-20,2010-02-26,2.56,211.3196,8.106,A,151315,8,2010,8.0,No Holiday,1409727.59
4,4,1,2010-02-27,2010-03-05,2.62,211.3501,8.106,A,151315,9,2010,8.0,No Holiday,1554806.68


In [185]:
walmart['Start_of_Week'] = pd.to_datetime(walmart['Start_of_Week'])
walmart = walmart.loc[ walmart['Start_of_Week'].dt.year == 2012 ]
walmart = walmart.reset_index()
walmart = walmart.drop(['index'], axis=1)
walmart

,ID,Store,Start_of_Week,Week_Date,Fuel_Price,CPI,Unemployment,Type,Size,Week,Year,Temperature_C,Holiday_Name,Weekly_Sales
0,101,1,2012-01-07,2012-01-13,3.26,219.8925,7.348,A,151315,2,2012,9.0,No Holiday,1459601.17
1,102,1,2012-01-14,2012-01-20,3.27,219.9857,7.348,A,151315,3,2012,12.0,No Holiday,1394393.84
2,103,1,2012-01-21,2012-01-27,3.29,220.0789,7.348,A,151315,4,2012,12.0,No Holiday,1319325.59
3,104,1,2012-01-28,2012-02-03,3.36,220.1720,7.348,A,151315,5,2012,14.0,No Holiday,1636339.65
4,105,1,2012-02-04,2012-02-10,3.41,220.2652,7.348,A,151315,6,2012,9.0,No Holiday,1802477.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,6434,45,2012-09-22,2012-09-28,4.00,192.0136,8.684,B,118221,39,2012,18.0,No Holiday,713173.95
1886,6435,45,2012-09-29,2012-10-05,3.98,192.1704,8.667,B,118221,40,2012,18.0,No Holiday,733455.07
1887,6436,45,2012-10-06,2012-10-12,4.00,192.3273,8.667,B,118221,41,2012,12.0,No Holiday,734464.36
1888,6437,45,2012-10-13,2012-10-19,3.97,192.3309,8.667,B,118221,42,2012,14.0,No Holiday,718125.53


In [186]:
for i in range(len(walmart)):
    walmart.loc[i, "Start_of_Week"] += dt.timedelta(days=1)
walmart

,ID,Store,Start_of_Week,Week_Date,Fuel_Price,CPI,Unemployment,Type,Size,Week,Year,Temperature_C,Holiday_Name,Weekly_Sales
0,101,1,2012-01-08,2012-01-13,3.26,219.8925,7.348,A,151315,2,2012,9.0,No Holiday,1459601.17
1,102,1,2012-01-15,2012-01-20,3.27,219.9857,7.348,A,151315,3,2012,12.0,No Holiday,1394393.84
2,103,1,2012-01-22,2012-01-27,3.29,220.0789,7.348,A,151315,4,2012,12.0,No Holiday,1319325.59
3,104,1,2012-01-29,2012-02-03,3.36,220.1720,7.348,A,151315,5,2012,14.0,No Holiday,1636339.65
4,105,1,2012-02-05,2012-02-10,3.41,220.2652,7.348,A,151315,6,2012,9.0,No Holiday,1802477.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,6434,45,2012-09-23,2012-09-28,4.00,192.0136,8.684,B,118221,39,2012,18.0,No Holiday,713173.95
1886,6435,45,2012-09-30,2012-10-05,3.98,192.1704,8.667,B,118221,40,2012,18.0,No Holiday,733455.07
1887,6436,45,2012-10-07,2012-10-12,4.00,192.3273,8.667,B,118221,41,2012,12.0,No Holiday,734464.36
1888,6437,45,2012-10-14,2012-10-19,3.97,192.3309,8.667,B,118221,42,2012,14.0,No Holiday,718125.53


In [189]:
walmart = walmart.filter(items=['Start_of_Week', 'Store', 'Fuel_Price', "CPI", "Unemployment", "Temperature_C"])
walmart

,Start_of_Week,Store,Fuel_Price,CPI,Unemployment,Temperature_C
0,2012-01-08,1,3.26,219.8925,7.348,9.0
1,2012-01-15,1,3.27,219.9857,7.348,12.0
2,2012-01-22,1,3.29,220.0789,7.348,12.0
3,2012-01-29,1,3.36,220.1720,7.348,14.0
4,2012-02-05,1,3.41,220.2652,7.348,9.0
...,...,...,...,...,...,...
1885,2012-09-23,45,4.00,192.0136,8.684,18.0
1886,2012-09-30,45,3.98,192.1704,8.667,18.0
1887,2012-10-07,45,4.00,192.3273,8.667,12.0
1888,2012-10-14,45,3.97,192.3309,8.667,14.0


In [191]:
X = walmart.drop("Start_of_Week", axis=1)
X = X.drop("Store", axis=1)
predix_scaled = X_scaler.transform(X)
dectre_labels = dectre.predict(predix_scaled)
dectre_labels = label_encoder.inverse_transform(dectre_labels)
dectre_labels

array(['Above', 'Below', 'Below', ..., 'Average', 'Average', 'Average'],
      dtype=object)

In [192]:
dectre_pre2012 = walmart
dectre_pre2012["Label"] = dectre_labels
dectre_pre2012

,Start_of_Week,Store,Fuel_Price,CPI,Unemployment,Temperature_C,Label
0,2012-01-08,1,3.26,219.8925,7.348,9.0,Above
1,2012-01-15,1,3.27,219.9857,7.348,12.0,Below
2,2012-01-22,1,3.29,220.0789,7.348,12.0,Below
3,2012-01-29,1,3.36,220.1720,7.348,14.0,Average
4,2012-02-05,1,3.41,220.2652,7.348,9.0,Above
...,...,...,...,...,...,...,...
1885,2012-09-23,45,4.00,192.0136,8.684,18.0,Average
1886,2012-09-30,45,3.98,192.1704,8.667,18.0,Average
1887,2012-10-07,45,4.00,192.3273,8.667,12.0,Average
1888,2012-10-14,45,3.97,192.3309,8.667,14.0,Average


In [193]:
dectre_predix = dectre_predix.rename(columns={'Start_of_Week': "Date", "Label": "Model_Label_Real"})

In [195]:
final_comparison = pd.merge(comparison, dectre_predix, how="outer", on=["Date", "Store", "Fuel_Price", "CPI", "Unemployment", "Temperature_C"])
final_comparison

,Date,Store,Fuel_Price,Pred_Fuel_Pric,CPI,Pred_CPI,Temperature_C,Pred_Temp,Unemployment,Pred_Unem,Weekly_Sales,Pred_Week_Sales,Ave_Sales,Proph_Label,Model_Label_Pred,Real_Label,Model_Label_Real
0,2012-01-08,1,3.26,3.110403,219.8925,219.979377,9.0,21.654107,7.348,7.758391,1459601.17,1.606048e+06,1.555264e+06,Average,Above,Average,Above
1,2012-01-15,1,3.27,3.090448,219.9857,220.190070,12.0,21.695015,7.348,7.748474,1394393.84,1.607312e+06,1.555264e+06,Average,Above,Average,Below
2,2012-01-22,1,3.29,3.070493,220.0789,220.400763,12.0,21.735924,7.348,7.738557,1319325.59,1.608577e+06,1.555264e+06,Average,Above,Below,Below
3,2012-01-29,1,3.36,3.050539,220.1720,220.611456,14.0,21.776833,7.348,7.728640,1636339.65,1.609841e+06,1.555264e+06,Average,Above,Average,Average
4,2012-02-05,1,3.41,3.030584,220.2652,220.822149,9.0,21.817742,7.348,7.718724,1802477.43,1.611106e+06,1.555264e+06,Average,Above,Above,Above
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1929,2012-09-23,45,4.00,2.718193,192.0136,194.449387,18.0,18.611209,8.684,8.197820,713173.95,8.620383e+05,7.859814e+05,Average,Average,Average,Average
1930,2012-09-30,45,3.98,2.700680,192.1704,194.587061,18.0,18.669415,8.667,8.190469,733455.07,8.628352e+05,7.859814e+05,Average,Average,Average,Average
1931,2012-10-07,45,4.00,2.683167,192.3273,194.724734,12.0,18.727620,8.667,8.183118,734464.36,8.636321e+05,7.859814e+05,Average,Average,Average,Average
1932,2012-10-14,45,3.97,2.665653,192.3309,194.862407,14.0,18.785825,8.667,8.175767,718125.53,8.644290e+05,7.859814e+05,Average,Average,Average,Average


In [196]:
final_comparison = final_comparison[["Date", "Store", "Fuel_Price", 'Pred_Fuel_Pric', 'CPI', 'Pred_CPI', 'Temperature_C', 'Pred_Temp', 'Unemployment', 'Pred_Unem', 'Weekly_Sales', "Pred_Week_Sales", 'Ave_Sales','Proph_Label', 'Model_Label_Pred', 'Model_Label_Real', 'Real_Label']]
final_comparison

,Date,Store,Fuel_Price,Pred_Fuel_Pric,CPI,Pred_CPI,Temperature_C,Pred_Temp,Unemployment,Pred_Unem,Weekly_Sales,Pred_Week_Sales,Ave_Sales,Proph_Label,Model_Label_Pred,Model_Label_Real,Real_Label
0,2012-01-08,1,3.26,3.110403,219.8925,219.979377,9.0,21.654107,7.348,7.758391,1459601.17,1.606048e+06,1.555264e+06,Average,Above,Above,Average
1,2012-01-15,1,3.27,3.090448,219.9857,220.190070,12.0,21.695015,7.348,7.748474,1394393.84,1.607312e+06,1.555264e+06,Average,Above,Below,Average
2,2012-01-22,1,3.29,3.070493,220.0789,220.400763,12.0,21.735924,7.348,7.738557,1319325.59,1.608577e+06,1.555264e+06,Average,Above,Below,Below
3,2012-01-29,1,3.36,3.050539,220.1720,220.611456,14.0,21.776833,7.348,7.728640,1636339.65,1.609841e+06,1.555264e+06,Average,Above,Average,Average
4,2012-02-05,1,3.41,3.030584,220.2652,220.822149,9.0,21.817742,7.348,7.718724,1802477.43,1.611106e+06,1.555264e+06,Average,Above,Above,Above
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1929,2012-09-23,45,4.00,2.718193,192.0136,194.449387,18.0,18.611209,8.684,8.197820,713173.95,8.620383e+05,7.859814e+05,Average,Average,Average,Average
1930,2012-09-30,45,3.98,2.700680,192.1704,194.587061,18.0,18.669415,8.667,8.190469,733455.07,8.628352e+05,7.859814e+05,Average,Average,Average,Average
1931,2012-10-07,45,4.00,2.683167,192.3273,194.724734,12.0,18.727620,8.667,8.183118,734464.36,8.636321e+05,7.859814e+05,Average,Average,Average,Average
1932,2012-10-14,45,3.97,2.665653,192.3309,194.862407,14.0,18.785825,8.667,8.175767,718125.53,8.644290e+05,7.859814e+05,Average,Average,Average,Average


In [197]:
final_comparison["Real_Label"].value_counts()

Average    1699
Below       106
Above        85
Name: Real_Label, dtype: int64

In [198]:
final_comparison["Model_Label_Real"].value_counts()

Average    1704
Above        99
Below        87
Name: Model_Label_Real, dtype: int64

In [199]:
final_comparison.to_csv("final_comparison.csv", index = False, header=True)

In [229]:
final_comparison = final_comparison[final_comparison['Date'] != '2012-01-01']

In [230]:
final_comparison[:50]

,Date,Store,Fuel_Price,Pred_Fuel_Pric,CPI,Pred_CPI,Temperature_C,Pred_Temp,Unemployment,Pred_Unem,Weekly_Sales,Pred_Week_Sales,Ave_Sales,Proph_Label,Model_Label_Pred,Model_Label_Real,Real_Label
0,2012-01-08,1,3.26,3.110403,219.8925,219.979377,9.0,21.654107,7.348,7.758391,1459601.17,1.606048e+06,1.555264e+06,Average,Above,Above,Average
1,2012-01-15,1,3.27,3.090448,219.9857,220.190070,12.0,21.695015,7.348,7.748474,1394393.84,1.607312e+06,1.555264e+06,Average,Above,Below,Average
2,2012-01-22,1,3.29,3.070493,220.0789,220.400763,12.0,21.735924,7.348,7.738557,1319325.59,1.608577e+06,1.555264e+06,Average,Above,Below,Below
3,2012-01-29,1,3.36,3.050539,220.1720,220.611456,14.0,21.776833,7.348,7.728640,1636339.65,1.609841e+06,1.555264e+06,Average,Above,Average,Average
4,2012-02-05,1,3.41,3.030584,220.2652,220.822149,9.0,21.817742,7.348,7.718724,1802477.43,1.611106e+06,1.555264e+06,Average,Above,Above,Above
5,2012-02-12,1,3.51,3.010630,220.4258,221.032842,7.0,21.858651,7.348,7.708807,1819870.00,1.612371e+06,1.555264e+06,Average,Above,Above,Above
6,2012-02-19,1,3.56,2.990675,220.6369,221.243536,14.0,21.899560,7.348,7.698890,1539387.83,1.613635e+06,1.555264e+06,Average,Above,Average,Average
7,2012-02-26,1,3.63,2.970720,220.8480,221.454229,16.0,21.940468,7.348,7.688973,1688420.76,1.614900e+06,1.555264e+06,Average,Above,Average,Average
8,2012-03-04,1,3.67,2.950766,221.0592,221.664922,15.0,21.981377,7.348,7.679056,1675431.16,1.616164e+06,1.555264e+06,Average,Above,Average,Average
9,2012-03-11,1,3.73,2.930811,221.2118,221.875615,18.0,22.022286,7.348,7.669140,1677472.78,1.617429e+06,1.555264e+06,Average,Above,Average,Average


In [231]:
final_comparison.to_csv("finalcomparison.csv", index = False, header=True)